In [1]:
import pandas as pd
import numpy as np

In [2]:
#df = pd.read_csv("./DATA/MACLAS_DATASET.csv")
#df = pd.read_excel("/home/dsg/MACLAS_final/intermediate_data/Marmota y Lapas.xlsx", engine='openpyxl') #directorio de trabajo
df = pd.read_excel("/home/dsg/MACLAS_final/DATA/mp-700.xlsx", engine='openpyxl') #directorio de trabajo


In [3]:

#df= df.drop_duplicates(subset=['Name'], keep='first')
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 184 columns):
 #    Column             Non-Null Count  Dtype         
---   ------             --------------  -----         
 0    id_inv             1 non-null      object        
 1    Method             1 non-null      object        
 2    Date XRF           1 non-null      datetime64[ns]
 3    Time               1 non-null      object        
 4    Duration           1 non-null      float64       
 5    Mg                 0 non-null      float64       
 6    +/-                0 non-null      float64       
 7    Al                 1 non-null      float64       
 8    +/-.1              1 non-null      float64       
 9    Si                 1 non-null      float64       
 10   +/-.2              1 non-null      float64       
 11   P                  1 non-null      float64       
 12   +/-.3              1 non-null      float64       
 13   S                  1 non-null      float64       
 1

 Performing a basic dimensional reduction by eliminating columns containing the "+/-" character.

In [4]:
#Dimensional reduction 
df = df.loc[:, ~df.columns.str.startswith('+/-')]
df.info(verbose=True, show_counts=True)    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 103 columns):
 #    Column             Non-Null Count  Dtype         
---   ------             --------------  -----         
 0    id_inv             1 non-null      object        
 1    Method             1 non-null      object        
 2    Date XRF           1 non-null      datetime64[ns]
 3    Time               1 non-null      object        
 4    Duration           1 non-null      float64       
 5    Mg                 0 non-null      float64       
 6    Al                 1 non-null      float64       
 7    Si                 1 non-null      float64       
 8    P                  1 non-null      float64       
 9    S                  1 non-null      float64       
 10   Cl                 1 non-null      float64       
 11   Ar                 0 non-null      float64       
 12   K                  1 non-null      float64       
 13   Ca                 1 non-null      float64       
 1

Then we create an intermediate dataframe filtering only the columns with information of our interest

In [5]:
contextual_info = df.iloc[:, :5]
filter_df = df.iloc[:, 5:]
filter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 98 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Mg                 0 non-null      float64
 1   Al                 1 non-null      float64
 2   Si                 1 non-null      float64
 3   P                  1 non-null      float64
 4   S                  1 non-null      float64
 5   Cl                 1 non-null      float64
 6   Ar                 0 non-null      float64
 7   K                  1 non-null      float64
 8   Ca                 1 non-null      float64
 9   Sc                 0 non-null      float64
 10  Ti                 1 non-null      float64
 11  V                  1 non-null      float64
 12  Cr                 1 non-null      float64
 13  Mn                 0 non-null      float64
 14  Fe                 1 non-null      float64
 15  Co                 0 non-null      float64
 16  Ni                 0 non-null 

Appling a 20% threshold of NaN values for delete columns

In [6]:
nan_treshold = 0.80  
cases = len(df) 

# iterate over each column of filter_df and eliminate those that exceed the limit of missing values
for col in filter_df.columns:
    missing_values = filter_df[col].isnull().sum()  # missinig values in each column
    missing_values_percentage = missing_values / cases  # precentage of NaN 

     #drop a column if NaN is greater than 20%
    if missing_values_percentage > nan_treshold:
        del filter_df[col]

In [6]:
filter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 81 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mg      31 non-null     float64
 1   Al      32 non-null     float64
 2   Si      32 non-null     float64
 3   P       32 non-null     float64
 4   S       32 non-null     float64
 5   Cl      32 non-null     float64
 6   Ar      0 non-null      float64
 7   K       21 non-null     float64
 8   Ca      32 non-null     float64
 9   Sc      0 non-null      float64
 10  Ti      31 non-null     float64
 11  V       1 non-null      float64
 12  Cr      27 non-null     float64
 13  Mn      22 non-null     float64
 14  Fe      32 non-null     float64
 15  Co      16 non-null     float64
 16  Ni      23 non-null     float64
 17  Cu      5 non-null      float64
 18  Zn      29 non-null     float64
 19  Ga      0 non-null      float64
 20  Ge      0 non-null      float64
 21  As      2 non-null      float64
 22  Se  

In [6]:
default_values = {
        'Mg': 0.01,'Al': 0.001,'Si': 0.001,
        'P': 0.001,'S': 0.001,'Cl': 0.001,
        'K': 0.0001,'Ca': 0.0001,'Sc': 0.0001,
        'Ti': 0.0001,'V': 0.0001,'Cr': 0.0001,
        'Mn': 0.0001,'Fe': 0.0001,'Co': 0.0001,
        'Ni': 0.0001,'Cu': 0.00001,'Zn': 0.00001,
        'Ga': 0.00001,'Ge': 0.00001,'As': 0.00001,
        'Se': 0.00001,'Br': 0.00001,'Rb': 0.00001,
        'Sr': 0.00001,'Y': 0.00001,'Zr': 0.00001,
        'Nb': 0.00001,'Mo': 0.00001,'Ru': 0.00001,
        'Pd': 0.00001,'Ag': 0.00001,'Cd': 0.00001,
        'In': 0.00001,'Sn': 0.00001,'Sb': 0.00001,
        'Te': 0.00001,'Ba': 0.00001,'Ta': 0.00001,
        'W': 0.00001,'Au': 0.00001,'Hg': 0.00001,
        'Tl': 0.00001,'Pb': 0.00001,'Th': 0.00001
    }
        


filter_df= filter_df.fillna(default_values)
#filter_df = filter_df.fillna(0,axis=1)
filter_df.head()

,Mg,Al,Si,P,S,Cl,Ar,K,Ca,Sc,...,Ancho / Diam,Ancho min,Largo / Grosor,Diam. Perforación,Peso,Type,L,A,B,Fecha cuaderno
0,0.01,22.2569,6.6711,16.8027,0.3207,4.64,NaN,0.7533,3.1145,0.0001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
filter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 98 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Mg                 1 non-null      float64
 1   Al                 1 non-null      float64
 2   Si                 1 non-null      float64
 3   P                  1 non-null      float64
 4   S                  1 non-null      float64
 5   Cl                 1 non-null      float64
 6   Ar                 0 non-null      float64
 7   K                  1 non-null      float64
 8   Ca                 1 non-null      float64
 9   Sc                 1 non-null      float64
 10  Ti                 1 non-null      float64
 11  V                  1 non-null      float64
 12  Cr                 1 non-null      float64
 13  Mn                 1 non-null      float64
 14  Fe                 1 non-null      float64
 15  Co                 1 non-null      float64
 16  Ni                 1 non-null 

In [8]:
filter_df = filter_df.dropna(axis=1)

In [12]:
filter_df = filter_df.fillna(0,axis=1)

In [9]:
filter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 45 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mg      1 non-null      float64
 1   Al      1 non-null      float64
 2   Si      1 non-null      float64
 3   P       1 non-null      float64
 4   S       1 non-null      float64
 5   Cl      1 non-null      float64
 6   K       1 non-null      float64
 7   Ca      1 non-null      float64
 8   Sc      1 non-null      float64
 9   Ti      1 non-null      float64
 10  V       1 non-null      float64
 11  Cr      1 non-null      float64
 12  Mn      1 non-null      float64
 13  Fe      1 non-null      float64
 14  Co      1 non-null      float64
 15  Ni      1 non-null      float64
 16  Cu      1 non-null      float64
 17  Zn      1 non-null      float64
 18  Ga      1 non-null      float64
 19  Ge      1 non-null      float64
 20  As      1 non-null      float64
 21  Se      1 non-null      float64
 22  Br    

Transforming values into atomic percentages by dividing each column value by the atomic weight of its element.

In [10]:
atomic_weights = {
        'Mg': 24.305,'Al': 26.982,'Si': 28.085,'P': 30.974,
        'S': 32.065,'Cl': 35.453,'K': 39.098,'Ca': 40.078,
        'Sc': 44.955,'Ti': 47.867,'V': 50.942,'Cr': 51.996,
        'Mn': 54.938,'Fe': 55.845,'Co': 58.933,'Ni': 58.693,
        'Cu': 63.546,'Zn': 65.380,'Ga': 69.723,'Ge': 72.63,
        'As': 74.922,'Se': 78.971,'Br': 79.904,'Rb': 85.468,
        'Sr': 87.620,'Y': 88.906,'Zr': 91.224,'Nb': 92.906,
        'Mo': 95.950,'Ru': 101.070,'Pd': 106.420,'Ag': 107.868,
        'Cd': 112.411,'In': 114.82,'Sn': 118.710,'Sb': 121.760,
        'Te': 127.600,'Ba': 137.327,'Ta': 180.948,'W': 183.840,
        'Au': 196.966,'Hg': 200.590,'Tl': 204.383,'Pb': 207.200,
        'Th': 232.038
    } 

# transform into atomic percentages________________________
for col, value in atomic_weights.items():
        try:
            filter_df[col] = filter_df[col] / value 
        except KeyError: # Ignoring if a column is not numeric
            pass

In [11]:
filter_df.K[17]

2.557675584428871e-06

In [14]:
filter_df = filter_df.round(5)

summing up the values of each sample and storing them in the sum variable

In [11]:
filter_df['Σ'] = np.sum(filter_df.iloc[:, :-1].values, axis=1)
filter_df.head()

,Mg,Al,Si,P,S,Cl,K,Ca,Sc,Ti,...,Te,Ba,Ta,W,Au,Hg,Tl,Pb,Th,Σ
0,0.000411,0.82488,0.237532,0.542478,0.010002,0.130877,0.019267,0.077711,0.000002,0.006019,...,7.836991e-08,0.001258,0.000301,5.439513e-08,5.077018e-08,4.985293e-08,0.000045,4.826255e-08,4.309639e-08,1.924169


In [16]:
filter_df.K[17]

0.14913

In [12]:

for col in filter_df.columns:
    if col != 'Σ':
        filter_df[col] = filter_df[col] / filter_df['Σ'] * 100
        

In [17]:
filter_df.K[17]

8.445342269680108

In [13]:
filter_df = filter_df.drop(['Σ'], axis=1)
filter_df.head()

,Mg,Al,Si,P,S,Cl,K,Ca,Sc,Ti,...,Sb,Te,Ba,Ta,W,Au,Hg,Tl,Pb,Th
0,0.021383,42.869383,12.344677,28.192818,0.519786,6.801766,1.001314,4.038676,0.000116,0.312798,...,0.000004,0.000004,0.065395,0.015624,0.000003,0.000003,0.000003,0.002339,0.000003,0.000002


In [14]:
filter_df['Σ'] = np.sum(filter_df.values, axis=1)
filter_df['Σ']=filter_df['Σ']
filter_df.head()

,Mg,Al,Si,P,S,Cl,K,Ca,Sc,Ti,...,Te,Ba,Ta,W,Au,Hg,Tl,Pb,Th,Σ
0,0.021383,42.869383,12.344677,28.192818,0.519786,6.801766,1.001314,4.038676,0.000116,0.312798,...,0.000004,0.065395,0.015624,0.000003,0.000003,0.000003,0.002339,0.000003,0.000002,100.000002


Merging preprocessed data with contextual information. 

In [15]:
preprocessed_df = pd.concat([contextual_info, filter_df], axis="columns")

preprocessed_df.head()

,id_inv,Method,Date XRF,Time,Duration,Mg,Al,Si,P,S,...,Te,Ba,Ta,W,Au,Hg,Tl,Pb,Th,Σ
0,MP-700,Soil_LE_FP,2014-10-28,15:05:44,60.532,0.021383,42.869383,12.344677,28.192818,0.519786,...,0.000004,0.065395,0.015624,0.000003,0.000003,0.000003,0.002339,0.000003,0.000002,100.000002


In [16]:
preprocessed_df.to_csv('./intermediate_data/mp-700_processed.csv')